In [1]:
import numpy as np
import time

from collections import namedtuple

In [2]:
'''
struct to storage order price and volume to trade

Usage:
a = Order(price=1, volume=1)
'''

Order = namedtuple('order', 'id price volume')

In [3]:
class dynamic_order:
    '''
    extension of Order, used in making deals
    '''
    def __init__(self, order):
        assert isinstance(order, Order)
        self.id = order.id
        self.price = order.price
        self.volume = order.volume
    @property
    def show(self):
        print('id : %d' % self.id)
        print('price : %d' % self.price)
        print('volume : %d' % self.volume)

In [4]:
class Trader:
    '''
    A primary class to define an investor
    '''
    def __init__(self, id):
        self.id = id
        self.cash = 0 # 现金
        self.cash_history = []
        self.asset = 0 # 总资产
        self.asset_history = []
        self.stock = 0 # 持股
        self.stock_history = []
        self.trade_history = [] # 交易记录
        self.price = 0
        self.order = Order(id=self.id,
                           price=0,
                           volume=0)
        
    def gen_order(self):
        self.order = Order(id=self.id,
                           price=np.random.randint(-10, 10), 
                           volume=np.random.randint(1, 5))
        
    def update(self, price, vol):
        vol *= np.sign(self.order.price)
        price = abs(price)
        self.cash -= price * vol
        self.cash_history.append(self.cash)
        self.stock += vol
        self.stock_history.append(self.stock)
        self.trade_history.append((price, vol))
        self.price = price
        self.asset = self.cash + self.stock * self.price
        self.asset_history.append(self.asset)
        
    @property
    def show(self):
        print('id : %d, cash : %d, stock : %d, stock_price : %d, asset : %d' % 
             (self.id, self.cash, self.stock, self.price, self.asset))
        
#         print('asset : %d' % self.asset)
#         print('stock : %d' % self.stock)
        

In [5]:
def make_deals(orders):
    buy_list = [dynamic_order(order) for order in orders if order.price>0]
    sell_list = [dynamic_order(order) for order in orders if order.price<0]

    buy_list = sorted(buy_list, key=lambda x:(-x.price, x.id))
    sell_list = sorted(sell_list, key=lambda x:(abs(x.price), x.id))
    deal_result = np.zeros(len(orders), int)
    price = 0
    if len(buy_list)==0 or len(sell_list)==0:
        return price, deal_result
    while sell_list[0].price+buy_list[0].price>=0:
        if sell_list[0].volume>=buy_list[0].volume:
            sell_list[0].volume -= buy_list[0].volume
            deal_result[buy_list[0].id] += buy_list[0].volume
            deal_result[sell_list[0].id] += buy_list[0].volume
            if sell_list[0].volume==buy_list[0].volume:
                price = (buy_list[0].price + abs(sell_list[0].price)) / 2
            else:
                price = abs(sell_list[0].price)
            _ = buy_list.pop(0)
        else:
            buy_list[0].volume -= sell_list[0].volume
            deal_result[buy_list[0].id] += sell_list[0].volume
            deal_result[sell_list[0].id] += sell_list[0].volume
            price = buy_list[0].price
            _ = sell_list.pop(0)           
        price = abs(price)
        if len(buy_list)==0 or len(sell_list)==0:
            break
    return price, deal_result

In [8]:
investors = [Trader(i) for i in range(20)]

In [9]:
orders = []
for investor in investors:
    investor.gen_order()
    orders.append(investor.order)
    
orders

[order(id=0, price=-10, volume=2),
 order(id=1, price=2, volume=4),
 order(id=2, price=-5, volume=3),
 order(id=3, price=9, volume=1),
 order(id=4, price=2, volume=1),
 order(id=5, price=8, volume=3),
 order(id=6, price=-8, volume=4),
 order(id=7, price=-3, volume=4),
 order(id=8, price=-6, volume=3),
 order(id=9, price=1, volume=2),
 order(id=10, price=7, volume=1),
 order(id=11, price=3, volume=1),
 order(id=12, price=-2, volume=1),
 order(id=13, price=1, volume=1),
 order(id=14, price=2, volume=2),
 order(id=15, price=-8, volume=4),
 order(id=16, price=-9, volume=4),
 order(id=17, price=6, volume=1),
 order(id=18, price=0, volume=2),
 order(id=19, price=-10, volume=3)]

In [10]:
start = time.time()
for step in range(3000):
    orders = []
    for investor in investors:
        investor.gen_order()
        orders.append(investor.order)

    price, deal_result = make_deals(orders)
    for i,investor in enumerate(investors):
        investor.update(price=price, vol=deal_result[i])
        
print('用时:', time.time()-start)

用时: 1.317028284072876


In [11]:
len(orders)

20

In [12]:
for investor in investors:
    investor.show

id : 0, cash : 512, stock : -113, stock_price : 4, asset : 60
id : 1, cash : 583, stock : -104, stock_price : 4, asset : 167
id : 2, cash : 562, stock : -101, stock_price : 4, asset : 158
id : 3, cash : -151, stock : -13, stock_price : 4, asset : -203
id : 4, cash : -380, stock : 98, stock_price : 4, asset : 11
id : 5, cash : 531, stock : -64, stock_price : 4, asset : 275
id : 6, cash : -305, stock : 55, stock_price : 4, asset : -85
id : 7, cash : -941, stock : 135, stock_price : 4, asset : -401
id : 8, cash : -522, stock : 60, stock_price : 4, asset : -282
id : 9, cash : 0, stock : 2, stock_price : 4, asset : 7
id : 10, cash : -428, stock : 76, stock_price : 4, asset : -124
id : 11, cash : 327, stock : -56, stock_price : 4, asset : 103
id : 12, cash : 76, stock : -46, stock_price : 4, asset : -107
id : 13, cash : -466, stock : 107, stock_price : 4, asset : -38
id : 14, cash : 98, stock : -21, stock_price : 4, asset : 14
id : 15, cash : -97, stock : -16, stock_price : 4, asset : -161
i